# Project 6 - Real-Time Anomaly Segmentation for Road Scenes


Clone of git repository

In [ ]:
# Installa Git se non è già disponibile (opzionale)
!apt-get install git

# Clona il repository (sostituisci il link con quello del tuo repository GitHub)
!rm -rf /content/AnomalySegmentation_CourseProjectBaseCode
!git clone https://github.com/FrancescoPassiatore/AnomalySegmentation_CourseProjectBaseCode.git

# Installa le dipendenze

!pip install ood-metrics


Fetch needed datasets from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Download of cityscapes dataset + generating labels

In [ ]:
!python -m pip install cityscapesscripts[gui] && csDownload
!mkdir /content/cityscapes
!csDownload gtFine_trainvaltest.zip -d /content/cityscapes
!csDownload leftImg8bit_trainvaltest.zip -d /content/cityscapes
!unzip /content/cityscapes/gtFine_trainvaltest.zip -d /content/cityscapes
!unzip /content/cityscapes/leftImg8bit_trainvaltest.zip -d /content/cityscapes
!CITYSCAPES_DATASET='/content/cityscapes/' csCreateTrainIdLabelImgs

# 2a - Evaluation (MSP, MaxLogit, MaxEntropy)

In [ ]:
import torch
import os

In [ ]:
for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for method in ['MSP','MaxLogit','MaxEntropy']:

    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - method : {method }")
    dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'

    !python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input {input} --method {method}  --loadDir {dir}

In [ ]:
#Evaluation of mIoU on Cityscapes dataset
!python /content/AnomalySegmentation_CourseProjectBaseCode/eval/eval_iou.py --datadir /content/cityscapes/ --subset val --loadDir /content/AnomalySegmentation_CourseProjectBaseCode/trained_models/ --loadWeights erfnet_pretrained.pth

# 2b - Evaluation temperature scaling

In [ ]:
for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for temperature in [0.1,0.25,0.5,0.75,1.1,1.5,2.0,2.5,3.0]:

    #Method is default MSP
    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - temperature : {temperature}")
    dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'

    !python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input {input} --temperature {temperature}  --loadDir {dir}

# Preparation for 3 - Training models

In [ ]:
import os
!pip install visdom

We modified every time the main function in /train. For better understanding in the GitHub repo everything is ordered in folders.

Training for Bisenet starting from a pretrained model found here : https://github.com/CoinCheung/BiSeNet

In [ ]:
model = 'bisenetv1'
dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/bisenetv1_pretrained.pth'
!cd "/content/AnomalySegmentation_CourseProjectBaseCode/train/" && python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/train/main.py --state {dir} --datadir '/content/cityscapes/' --model {model} --savedir '/content/training_output_bisenet/' --num-epochs=20 --epochs-save=1

Training for ENet starting from a pretraned model found here : https://github.com/davidtvs/PyTorch-ENet

In [ ]:
model = 'enet'
dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/ENet'
!cd "/content/AnomalySegmentation_CourseProjectBaseCode/train/" && python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/train/main.py --state {dir} --datadir '/content/cityscapes/' --model {model} --savedir '/content/training_output_enet/' --num-epochs=20 --epochs-save=1

Training for ERFNet startng from initial repo

In [ ]:
model = 'erfnet'
dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/erfnet_pretrained.pth'
!cd "/content/AnomalySegmentation_CourseProjectBaseCode/train/" && python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/train/main.py --state {dir} --datadir '/content/cityscapes/' --model {model} --savedir '/content/training_output_erfnet/' --num-epochs=20 --epochs-save=1

# 3 - Void classification

Three different evaluations for the 3 models. We load the weights from the respective model_best.pth generated from each training.

In [ ]:
for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for method in ['Void']:

    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - method : {method}")
    dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'
    !python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input {input} --loadModel 'enet.py' --method {method} --loadDir {dir} --loadWeights 'model_best.pth'


In [ ]:
for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for method in ['Void']:

    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - method : {method}")
    dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'
    !python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input {input} --loadModel 'bisenetv1.py' --method {method} --loadDir {dir} --loadWeights 'model_best.pth'

In [ ]:
for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for method in ['Void']:

    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - method : {method}")
    dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'
    !python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input {input} --loadModel 'erfnet.py' --method {method} --loadDir {dir} --loadWeights 'model_best.pth'

Evaluation of mIoU

In [ ]:
!python /content/AnomalySegmentation_CourseProjectBaseCode/eval/eval_iou.py --datadir /content/cityscapes/ --loadModel erfnet.py --subset val --loadDir /content/AnomalySegmentation_CourseProjectBaseCode/trained_models/ --loadWeights model_best.pth

In [ ]:
!python /content/AnomalySegmentation_CourseProjectBaseCode/eval/eval_iou.py --datadir /content/cityscapes/ --loadModel bisenetv1.py --subset val --loadDir /content/AnomalySegmentation_CourseProjectBaseCode/trained_models/ --loadWeights model_best.pth

In [ ]:
!python /content/AnomalySegmentation_CourseProjectBaseCode/eval/eval_iou.py --datadir /content/cityscapes/ --loadModel enet.py --subset val --loadDir /content/AnomalySegmentation_CourseProjectBaseCode/trained_models/ --loadWeights model_best.pth

# 4 -  Effect of Training Loss function

As for the training of models we modified every time the main.py. The correct implementation is in the repo under the respective folder.

Training erfnet with IsoMaxPlus (CE alreasy included in secondpart of formula)

In [ ]:
model = 'erfnet'
dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/erfnet_pretrained.pth'
!cd "/content/AnomalySegmentation_CourseProjectBaseCode/train/" && python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/train/main.py  --state {dir} --datadir '/content/cityscapes/' --model {model} --savedir '/content/training_output_erfnet_w_isomax_plus/' --num-epochs=20 --epochs-save=1

Training erfnet with IsoMaxPlus and FocalLoss

In [ ]:
model = 'erfnet'
dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/erfnet_pretrained.pth'
!cd "/content/AnomalySegmentation_CourseProjectBaseCode/train/" && python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/train/main.py  --state {dir} --datadir '/content/cityscapes/' --model {model} --savedir '/content/training_output_erfnet_w_isomax_plus_focal/' --num-epochs=20 --epochs-save=1

Evaluation made for both IsoMaxPlus with and without FocalLoss . In model_best.pth the trained models.

In [ ]:

for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for method in ['MSP','MaxLogit','MaxEntropy']:


    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - method : {method}")
    dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'
    !python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input {input} --method {method} --loadWeights model_best.pth --loadDir {dir}


Evaluation of mIoU for both IsoMaxPLus with and without FocalLoss.

In [ ]:
!python /content/AnomalySegmentation_CourseProjectBaseCode/eval/eval_iou.py --datadir /content/cityscapes/ --loadModel ernet.py --subset val --loadDir /content/AnomalySegmentation_CourseProjectBaseCode/trained_models/ --loadWeights model_best.pth

Training erfnet with LogitNorm

In [ ]:
import os
!pip install visdom

model = 'erfnet'
dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/erfnet_pretrained.pth'
!cd "/content/AnomalySegmentation_CourseProjectBaseCode/train/" && python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/train/train_efnet_logit_norm/erfnet_logit_norm.py --state {dir} --datadir '/content/cityscapes/' --model {model} --savedir '/content/logitNorm_CrossEntropy/' --num-epochs=20 --epochs-save=1

Training erfnet with LogitNorm and FocalLoss

In [ ]:
import os
!pip install visdom

model = 'erfnet'
dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/erfnet_pretrained.pth'
!cd "/content/AnomalySegmentation_CourseProjectBaseCode/train/" && python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/train/train_erfnet_logit_norm_focal/erfnet_focal_logit.py --state {dir} --datadir '/content/cityscapes/' --model {model} --savedir '/content/training_output_logitnorm/' --num-epochs=20 --epochs-save=1

Evaluation made for both LogitNorm with and without FocalLoss . In model_best.pth the trained models.

In [ ]:
for dataset in ['RoadAnomaly21','RoadObsticle21','fs_static','FS_LostFound_full','RoadAnomaly']:
  for method in ['MSP','MaxLogit','MaxEntropy']:


    format_file = os.listdir(f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images')[0].split(".")[1]
    input = f'/content/drive/MyDrive/Validation_Dataset/{dataset}/images/\*.{format_file}'
    print(f"\nDataset: {dataset} - method : {method}")
    dir = f'/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'
    !python -W ignore /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input {input} --method {method} --loadWeights model_best.pth --loadDir {dir}

Evaluation of mIoU for both LogitNormalization with and without FocalLoss.

In [ ]:
!python /content/AnomalySegmentation_CourseProjectBaseCode/eval/eval_iou.py --datadir /content/cityscapes/ --loadModel ernet.py --subset val --loadDir /content/AnomalySegmentation_CourseProjectBaseCode/trained_models/ --loadWeights model_best.pth